# VGG16 Feature Extractors CIFAR10

In [ ]:
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear  
# inline in the notebook rather than in a new window.
%matplotlib inline

# set default size of plots
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# import the necessary packages
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from utils.hdf5_dataset_writer import HDF5DatasetWriter
import numpy as np
import progressbar
import random
import os

from utils.image_preprocessor import ImagePreprocessor
from utils.image_preprocessor import ResizePreprocessor
from utils.image_preprocessor import ImageToArrayPreprocessor

In [ ]:
from keras.datasets import cifar10
((trainX, trainY), (testX, testY)) = cifar10.load_data()

In [ ]:
sample_num = 1000

In [ ]:
trainX = trainX[:sample_num]
trainX.shape

In [ ]:
trainY = trainY[:sample_num]
trainY.shape

In [ ]:
aap = ResizePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()
ip = ImagePreprocessor(preprocessors=[aap, iap])

In [ ]:
trainX = ip.preprocess(trainX)

In [ ]:
plt.imshow(trainX[0])

In [ ]:
# path to output HDF5 file
output = './output/feature_persistence/vgg16-cifar10-features.hdf5'
# batch size of images to be passed through network, default=32
batch_size = 32
# size of feature extraction buffer, default=1000
buffer_size = 1000

In [ ]:
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
# load the VGG16 network
print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)

# initialize the HDF5 dataset writer, then store the class label names in the dataset
dataset = HDF5DatasetWriter((sample_num, 512 * 7 * 7), output, dataKey="features", bufSize=buffer_size) 
dataset.storeClassLabels(labelNames)

In [ ]:
# initialize the progress bar
widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=sample_num, widgets=widgets).start()

# loop over the images in patches
for i in np.arange(0, sample_num, batch_size):
    # extract the batch of images and labels, then initialize the
    # list of actual images that will be passed through the network
    # for feature extraction
    batchImages = trainX[i:i + batch_size]
    batchLabels = trainY[i:i + batch_size]
    batchLabels = np.squeeze(batchLabels)
    
    # subtracting the mean RGB pixel intensity from the ImageNet dataset
    batchImages = imagenet_utils.preprocess_input(batchImages)
    
    # pass the images through the network and use the outputs as our actual features
    features = model.predict(batchImages, batch_size=batch_size)
    
    # reshape the features so that each image is represented by
    # a flattened feature vector of the ‘MaxPooling2D‘ outputs
    features = features.reshape((features.shape[0], 512 * 7 * 7))

    # add the features and labels to our HDF5 dataset
    dataset.add(features, batchLabels)
    pbar.update(i)
    
# close the dataset
dataset.close()
pbar.finish()

In [ ]:
import h5py
db = h5py.File(output)
list(db.keys())

# Training a Classifier on Extracted Features CIFAR10

In [ ]:
# import the necessary packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import argparse
import pickle
import h5py

In [ ]:
# construct the arguments
# path HDF5 database
output = './output/feature_persistence/vgg16-cifar10-features.hdf5'
# path to output model
model_path = './output/feature_persistence/vgg16-cifar10-model.cpickle'
# num of jobs to run when tuning hyperparameters
jobs = -1

In [ ]:
# open the HDF5 database for reading then determine the index of
# the training and testing split, provided that this data was
# already shuffled *prior* to writing it to disk
db = h5py.File(output, "r")
i = int(db["labels"].shape[0] * 0.75)

# define the set of parameters that we want to tune then start a
# grid search where we evaluate our model for each value of C
print("[INFO] tuning hyperparameters...")
params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}
model = GridSearchCV(LogisticRegression(), params, cv=3, n_jobs=jobs)
model.fit(db["features"][:i], db["labels"][:i])
print("[INFO] best hyperparameters: {}".format(model.best_params_))

# evaluate the model
print("[INFO] evaluating...")
preds = model.predict(db["features"][i:])
print(classification_report(db["labels"][i:], preds, target_names=db["label_names"]))

In [ ]:
# serialize the model to disk
print("[INFO] saving model...")
f = open(model_path, "wb")
f.write(pickle.dumps(model.best_estimator_))
f.close()

# close the database
db.close()

Well Done